In [22]:
# IMPORTS & UNITS

#---------------------------------------
import numpy as np
from matplotlib import pyplot as plt
from astropy import constants as astro_const

#---------------------------------------------
# Animation libraries
import matplotlib.animation as animation
from IPython.display import HTML

In [23]:
# INPUT READING FUNCTIONS

#---------------------------------------------
# Read the input file:
#
# N = number of particles
# t_0 = initial time
# particles_0 = collection of mass + 6D position/velocity vector for every particle in the system
# The shape of particles is: [[m_1 x_1, y_1, z_1, vx_1, vy_1, vz_1], 
#                             [m_2 x_2, y_2, z_2, vx_2, vy_2, vz_2],
#                             ...,
#                             [m_N x_N, y_N, z_N, vx_N, vy_N, vz_N]]
#
# Returns:
#         N
#         a snapshot ("snapshot_0") of the system at the initial condition in the shape:
#         [N, t_0, particles_0] :   [N, t_0, [[m_1 x_1, y_1, z_1, vx_1, vy_1, vz_1], 
#                                             [m_2 x_2, y_2, z_2, vx_2, vy_2, vz_2],
#                                             ...,
#                                             [m_N x_N, y_N, z_N, vx_N, vy_N, vz_N]]]
#
# Input file is structured as follow:
#
# N
# t_0
# m_1 x_1, y_1, z_1, vx_1, vy_1, vz_1
# m_2 x_2, y_2, z_2, vx_2, vy_2, vz_2
# ...
# m_N x_N, y_N, z_N, vx_N, vy_N, vz_N
def read_input(input_file_path):
    input_file = open(input_file_path)

    # readline() reads the first line and returns it as a string
    N = int(input_file.readline())
    t_0 = float(input_file.readline())

    # readlines() reads all the remaining lines and returns them into a list of strings
    particles_0 = input_file.readlines()

    input_file.close()

    # Convert the list of strings into a list of lists of number
    for i in range(len(particles_0)):
        # split() splits a string based on a separator (default is ' ') and returns a list of strings
        particles_0[i] = particles_0[i].split()

        # Convert the elements of each splitted string into numbers
        for j in range(len(particles_0[i])):
            particles_0[i][j] = float(particles_0[i][j])

    snapshot_0 = [N, t_0, particles_0]

    return snapshot_0

#-----------------------------------------------------------------------------
# Read the output file:
# # Output file is structured as follow:
#
# N
# t_1
# m_1 x_1, y_1, z_1, vx_1, vy_1, vz_1
# m_2 x_2, y_2, z_2, vx_2, vy_2, vz_2
# ...
# m_N x_N, y_N, z_N, vx_N, vy_N, vz_N
# N
# t_2
# m_1 x_1, y_1, z_1, vx_1, vy_1, vz_1
# m_2 x_2, y_2, z_2, vx_2, vy_2, vz_2
# ...
# m_N x_N, y_N, z_N, vx_N, vy_N, vz_N
#
# ...
#
# N
# t_f
# m_1 x_1, y_1, z_1, vx_1, vy_1, vz_1
# m_2 x_2, y_2, z_2, vx_2, vy_2, vz_2
# ...
# m_N x_N, y_N, z_N, vx_N, vy_N, vz_N
#
# Returns a list of snapshot of the evolved system (t>t_0)
# See read_input for details on the snapshots
def read_output(output_file_path):
    output_file = open(output_file_path)

    # List to store all the snapshots in the output file
    snapshots = []

    # Try except here detects the EOF
    # int() throws an exception when tries to convert something different than a number
    while True:
        try:
            N = int(output_file.readline())
        except:
            break

        # Line after N is the time stamp of the snapshot
        t_stamp = float(output_file.readline())

        # Collect strings of mass + 6D position/velocity vector N times (for N particles)
        particles = [output_file.readline() for i in range (N)]

        # Convert the list of strings into a list of lists of number (same as for read_input())
        for i in range(len(particles)):
            particles[i] = particles[i].split()

            for j in range(len(particles[i])):
                particles[i][j] = float(particles[i][j])
        
        # Append the snapshot [t_stamp, particles] to the snapshots list
        snapshots.append([N, t_stamp, particles])

    output_file.close()
    
    return snapshots

#--------------------------------------------------------------------------
# Returns:
#         N
#         The complete list of snapshots of the system (including the initial one)
#
# How to navigate the snapshots:
# 1) Choose the snapshot:
#    *** snapshots[snapshot_index, from 0 to len(snapshot)-1] ***
#
# 2) What do you want?
#    A) Number of particles:
#       *** snapshot[snapshot_index][0] ***
#       Note that snapshot[snapshot_index][0] can't be subscripted anymore
#
#    B) Time stamp: 
#       *** snapshot[snapshot_index][1] ***
#       Note that snapshot[snapshot_index][1] can't be subscripted anymore
#
#    C) Particles masses, positions and velocities:
#       *** snapshot[snapshot_index][2] ***
#       This is a list of lists (see read_input() for details)
#
# 3) Which particle are you interested in?
#    *** snapshot[snapshot_index][2][particle_index, from 0 to N-1] ***
#
# 4) What do you want to know about the particle:
#    *** Mass: snapshot[snapshot_index][2][particle_index][0] ***
#    *** x, y, z: snapshot[snapshot_index][2][particle_index][1, 2, 3] ***
#    *** vx, vy, vz: snapshot[snapshot_index][2][particle_index][1+3, 2+3, 3+3] ***
def read_snapshots(input_file_path, output_file_path):
    snapshot_0 = read_input(input_file_path)
    snapshots = read_output(output_file_path)

    # Append the intial snapshot to the beginning of the list
    snapshots = [snapshot_0] + snapshots

    return snapshots

In [24]:
# MATHEMATICAL COMPUTATIONAL FUNCTIONS

#-------------------------------------------------------------------------
# Returns the difference between two vectors (vector_2 - vector_1)
# 
# If vectors are positions:
# It is a vector that points from position_1 to position_2: [x_2 - x_1, y_2 - y_1, z_2 - z_1]
# 
# If vectors are velocities:
# It is a vectors that describes the relative velocity in the frame in which particle 1 is at rest
# [vx_2 - vx_1, vy_2 - vy_1, vz_2 - vz_1]
#
# Imput parameters are strictly numpy arrays of 3 numbers: (x, y, z)
def compute_vector_diff(vector_1, vector_2):
    return vector_2 - vector_1

# Returns the vectorial product of two vectors
# Imput parameters are strictly numpy arrays of 3 numbers: (x, y, z)
def compute_vectorial_product(vector_1, vector_2):
    x = vector_1[1] * vector_2[2] - vector_1[2] * vector_2[1]
    y = vector_1[2] * vector_2[0] - vector_1[0] * vector_2[2]
    z = vector_1[0] * vector_2[1] - vector_1[1] * vector_2[0]

    return np.array([x, y, z])

# Returns the absolute value of a vector to the absval-th power
def compute_vector_abs(vector, absval=1):
    value = 0

    for i in range(len(vector)):
        value += vector[i] ** 2

    value = value ** (1 / 2)

    return value ** absval

# Returns the distance between two points
# Imput parameters are arrays of 3 numbers: (x, y, z)
def compute_distance(point_1, point_2):
    distance = 0
    for i in range (3):
        distance += (point_1[i] - point_2[i]) ** 2
    
    return distance ** (1 / 2)

In [25]:
# SNAPSHOTS READING FUNCTIONS & SMALL CALCULATIONS

#------------------------------------------------------------------
# Returns an array of the number of particles of the snapshots
def get_N(snapshots):
    return np.array([snapshots[i][0] for i in range(len(snapshots))])

# Returns an array of the time stamps of the snapshots
def get_timestamps(snapshots):
    return np.array([snapshots[i][1] for i in range(len(snapshots))])

# Returns an array of the masses of the specified snapshot
def get_masses(snapshots, snapshot_index=0, total=False):
    if not total:
        masses = np.array([snapshots[snapshot_index][2][i][0]
                           for i in range(len(snapshots[snapshot_index][2]))])
    elif total:
        masses = np.sum([snapshots[snapshot_index][2][i][0]
                         for i in range(len(snapshots[snapshot_index][2]))])
        
    return masses

# Compute the position or velocity of the center of mass for the specified snapshot
# Returns an array of 3 values: [qt_x, qt_y, qt_z]
# vec = 'pos', 'vel'
def get_center_of_mass(snapshots, snapshot_index=0, vec=''):
    M_tot = np.sum(get_masses(snapshots, snapshot_index))
    qtX_CM, qtY_CM, qtZ_CM = 0, 0, 0

    if vec.lower() == 'pos':
        shift = 0
    elif vec.lower() == 'vel':
        shift = 3

    for i in range(snapshots[snapshot_index][0]):
        qtX_CM += snapshots[snapshot_index][2][i][0] * snapshots[snapshot_index][2][i][1+shift]
        qtY_CM += snapshots[snapshot_index][2][i][0] * snapshots[snapshot_index][2][i][2+shift]
        qtZ_CM += snapshots[snapshot_index][2][i][0] * snapshots[snapshot_index][2][i][3+shift]
    
    qtX_CM /= M_tot
    qtY_CM /= M_tot
    qtZ_CM /= M_tot

    return np.array([qtX_CM, qtY_CM, qtZ_CM])

# absval = 0: returns an array of 3D vectors of the specified particle, for the specified quantity
#             in the specified reference frame
# [[Qx1_0, Qy_0, Qz_0], [Qx_1, Qy_1, Qz_1], ..., [len(snapshots)-1]]
# absval = n != 0: returns the absolute value of the vectors to the absval-th power
# vec = 'pos', 'vel'
# rf = 'snapshot', 'cm' (center of mass), 'particle' (specify which particle with the next parameter)
def get_3D_vec(snapshots, particle_index, vec='', absval=0, rf='snapshot', rf_particle_index=None):
    if vec.lower() == 'pos':
        shift = 0
    elif vec.lower() == 'vel':
        shift = 3
    
    # Compute in the snapshot reference frame
    if rf.lower() == 'snapshot':
        result = np.array([snapshots[i][2][particle_index][1+shift:4+shift]
                           for i in range(len(snapshots))])
    
    # Compute in the center of mass reference frame
    elif rf.lower() == 'cm':
        result = compute_vector_diff(np.array([get_center_of_mass(snapshots, i, vec)
                                               for i in range(len(snapshots))]),
                                               np.array([snapshots[i][2][particle_index][1+shift:4+shift]
                                                         for i in range(len(snapshots))]))
    
    # Compute in the reference frame of the specified particle
    elif rf.lower() == 'particle':
        result = compute_vector_diff(np.array([snapshots[i][2][rf_particle_index][1+shift:4+shift]
                                               for i in range(len(snapshots))]),
                                               np.array([snapshots[i][2][particle_index][1+shift:4+shift]
                                                         for i in range(len(snapshots))]))

    # Return the result as it is
    if absval == 0:
        pass
    
    # Compute the absval-th power of every vector into "result"
    elif absval != 0:
        for i in range(len(result)):
            result[i] = compute_vector_abs(result[i], absval)

        # Because of how numpy arrays works, we need to slice the array to take the first column
        # Actually any column would be fine because now their content is equal
        # The point is that we want just one set of values and not three
        result = result[:, :1]

        # Slicing returns an array with shape [[x_0], [x_1], ..., [x_(len(snapshot)-1)]]
        # Redefine the array to get rid of the 1-element lists
        result = np.array([result[i][0] for i in range(len(result))])

    return result

# Returns an array of the specified coordinate for the specified particle for the specified quantity
# in the specified reference frame
# vec = 'pos', 'vel'
# axe = 'x', 'y', 'z'
# rf = 'snapshot', 'cm' (center of mass), 'particle' (specify which particle with the next parameter)
def get_coord(snapshots, particle_index, vec='', axe='', rf='snapshot', rf_particle_index=None):
    if axe.lower() == 'x':
        coordinates = get_3D_vec(snapshots, particle_index, vec,
                                 rf=rf, rf_particle_index=rf_particle_index)[:, :1]
    elif axe.lower() == 'y':
        coordinates = get_3D_vec(snapshots, particle_index, vec,
                                 rf=rf, rf_particle_index=rf_particle_index)[:, 1:2]
    elif axe.lower() == 'z':
        coordinates = get_3D_vec(snapshots, particle_index, vec,
                                 rf=rf, rf_particle_index=rf_particle_index)[:, 2:3]

    return np.array([coordinates[i][0] for i in range(len(coordinates))])

In [26]:
# PHYSICS COMPUTATIONAL FUNCTIONS

#-------------------------------------------------------------------------
# Returns the energy of the system in the specified reference frame
# E_type = 'kin', 'pot', 'tot'
# rf = 'snapshot', 'cm' (center of mass)
def get_energy(snapshots, snapshot_index, E_type='', rf='snapshot', rf_particle_index=None):
    N = snapshots[snapshot_index][0]

    if E_type.lower() == 'kin':
        E = 0
        # A list of lists of the velociies squared of every massess at every time
        # [[vsq_t0_0, vsq_t1_0, ..., vsq_tf_0], [vsq_t0_1, vsq_t1_1, ..., vsq_tf_1], ..., [vsq_t_(N-1)]]
        v_squared = [get_3D_vec(snapshots, i, 'vel', 2, rf, rf_particle_index) for i in range (N)]

        for i in range(N):
            E += (1 / 2) * snapshots[snapshot_index][2][i][0] * v_squared[i][snapshot_index]
    elif E_type.lower() == 'pot':
        E = 0
        # Same as above but it's for position vectors
        # positions[i][snapshot_index] will be the 3D vector of the i-th particle at specified snapshot
        positions = [get_3D_vec(snapshots, i, 'pos', 0, rf, rf_particle_index) for i in range(N)]

        # Cycle over all possible combination of particles
        # part_0: part_1, part_2, part_3, ..., part_(N-1)
        # part_1: part_2, part_3, part_4, ..., part_(N-1)
        # part_2: part_3, part_4, part_5, ..., part_(N-1)
        # ...
        # part_i: part_(i+1), part_(i+2), ..., part_(N-1)
        for i in range(N):
            for j in range(i+1, N):
                distance = compute_distance(positions[i][snapshot_index], positions[j][snapshot_index])
                E += -1 * snapshots[snapshot_index][2][i][0] * snapshots[snapshot_index][2][j][0] / distance
    elif E_type.lower() == 'tot':
        E_kin = get_energy(snapshots, snapshot_index, 'kin', rf, rf_particle_index)
        E_pot = get_energy(snapshots, snapshot_index, 'pot', rf, rf_particle_index)
        E = E_kin + E_pot

    return E

In [27]:
# GLOBAL VARIABLES

#---------------------------------------------
# Relevant constants in CGS
G = astro_const.G.cgs
M_sun = astro_const.M_sun.cgs
kpc = astro_const.kpc.cgs

# Internal units conversion factors: G = 1, M = 1 M_sun, r = 1 kpc
v_I_to_v_cgs = (G * M_sun / kpc) ** (1 / 2)
t_I_to_t_cgs = kpc / v_I_to_v_cgs

# Strings for units
mass_unit_str = "$M_\odot$"
distance_unit_str = "$kpc$"

# Printing conversion factors
print(v_I_to_v_cgs)
print(t_I_to_t_cgs)

207.38652969844205 cm / s
1.4878871766542453e+19 s


<>:14: SyntaxWarning: invalid escape sequence '\o'
<>:14: SyntaxWarning: invalid escape sequence '\o'
/var/folders/05/2fbv44555hx420qclxnfbd4c0000gn/T/ipykernel_82975/487013095.py:14: SyntaxWarning: invalid escape sequence '\o'
  mass_unit_str = "$M_\odot$"


In [28]:
# # FILE READING

# #----------------------------------------------------------------
# input_file_path = "input_3.txt"
# output_file_path = "output_3.txt"

# snapshots = read_snapshots(input_file_path, output_file_path)

# pos = np.array([[get_coord(snapshots, i, 'pos', 'x', rf='snapshot'), get_coord(snapshots, i, 'pos', 'y', rf='snapshot')] for i in range (2)])
# vel = np.array([[get_coord(snapshots, i, 'vel', 'x', rf='snapshot'), get_coord(snapshots, i, 'vel', 'y', rf='snapshot')] for i in range (2)])
# # pos = np.array([[get_coord(snapshots, i, 'pos', 'x', rf='cm'), get_coord(snapshots, i, 'pos', 'y', rf='cm')] for i in range (2)])
# # vel = np.array([[get_coord(snapshots, i, 'vel', 'x', rf='cm'), get_coord(snapshots, i, 'vel', 'y', rf='cm')] for i in range (2)])
# cm_pos = np.array([get_center_of_mass(snapshots, i, 'pos') for i in range(len(snapshots))])
# time_stamp = get_timestamps(snapshots)